## Custom QC for motion energy

In [1]:
prefix = '/home/ines/repositories/'
prefix = '/Users/ineslaranjeira/Documents/Repositories/'

In [2]:
"""
Check sessions available with dlc data
@author: Ines
"""
import pandas as pd
import pickle 
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np
from one.api import ONE
from uuid import UUID
import json
from scipy.stats import zscore

# Get my functions
functions_path =  prefix + '/representation_learning_variability/Functions/'
os.chdir(functions_path)
from video_functions import get_ME
from one_functions_generic import timeseries_PSTH

one = ONE(mode='remote')

### Get extended QC of all sessions

In [ ]:
ext_qc_path = prefix + '/representation_learning_variability/Video and wheel/Video QC/'
os.chdir(ext_qc_path)
ext_qc = pickle.load(open(ext_qc_path + "extended_qc07-08-2025", "rb"))
use_qc = ext_qc.loc[ext_qc['task'].isin(['PASS', 'WARNING'])]
final_qc = use_qc.loc[(use_qc['_lightningPoseLeft_lick_detection'].isin(['PASS'])) &
                             (use_qc['_lightningPoseLeft_time_trace_length_match'].isin(['PASS'])) &   
                             (use_qc['_videoLeft_pin_state'].apply(lambda x: (isinstance(x, list) and True in x) or x == 'PASS')) &
                             (use_qc['_lightningPoseLeft_trace_all_nan'].isin(['PASS'])) &
                             (use_qc['_videoLeft_framerate'].apply(lambda x: (isinstance(x, list) and True in x) or x == 'PASS')) &   
                             (use_qc['_videoLeft_camera_times'].apply(lambda x: (isinstance(x, list) and True in x) or x == 'PASS')) &   
                             (use_qc['_videoLeft_dropped_frames'].apply(lambda x: (isinstance(x, list) and True in x) or x == 'PASS')) &  # can make more conservative by removing or  x == None
                             (use_qc['_videoLeft_timestamps'].isin([True, 'PASS']))                   
                            ]

In [ ]:
video_type = 'left'
save_path = prefix + '/representation_learning_variability/Video and wheel/Video QC/Figures/ME/'
files = os.listdir(save_path)

eids = final_qc['eid'].unique()
for e, session in enumerate(eids):
        filename = str(session + 'whisker_psth_-12_feedback.png')
        if filename not in files:
            try:
                me_times, motion_energy = get_ME(one, session, video_type)       
                m = zscore(motion_energy,nan_policy='omit') 
                
                t_init = 1
                t_end = 2
                event = 'stimOn_times'
                event = 'feedback_times'

                session_trials = one.load_object(session, obj='trials', namespace='ibl')
                session_trials = session_trials.to_df() 
                
                # Align data for PSTH
                ME_left = timeseries_PSTH(me_times, m, session_trials, event, t_init, t_end, subtract_baseline=False)
                
                # Plot and save

                fig, ax = plt.subplots(nrows=1, ncols=1, sharex=True, sharey=True, figsize=[7, 5])
                plt.rc('font', size=12)

                this_data = ME_left.reset_index()
                this_data['smooth_value'] = this_data['value'].rolling(3, min_periods=1).mean()
                this_data = this_data.loc[this_data['choice'].isin([1., -1.])]
                sns.lineplot(x='variable', y='smooth_value', style='feedback', data=this_data, ax = ax)
                #plt.hlines(0, -0.5, 1, color='gray', linestyles='--')
                plt.vlines(0, -.75, 1.5, color='gray', linestyles='--')
                ax.set_xlabel(' ')
                ax.set_xlabel('Time from ' + event + ' (s)')
                ax.set_ylabel('z-scored motion energy')
                ax.set_title(session)
                plt.tight_layout()

                # Save the plot as a PNG file
                plt.savefig(str(save_path + session + 'whisker_psth_-12_feedback.png'), format='png')
                # Display the plot
                # plt.show()
            except:
                 print(session)
                        

/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/api.py:291: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  self._cache[table].loc[records.index[to_update], :] = records[to_update]
/opt/anaconda3/envs/iblenv/lib/python3.

3dd347df-f14e-40d5-9ff2-9c49f84d2157


/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/one_functions_generic.py:554: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  trials['prev_choice'][1:] = trials['choice'][:-1]
/Users/ineslaranjeira/Do

3c851386-e92d-4533-8d55-89a46f0e7384


/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/one_functions_generic.py:554: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  trials['prev_choice'][1:] = trials['choice'][:-1]
/Users/ineslaranjeira/Do

...

100%|██████████| 0.00787353515625/0.00787353515625 [00:00<00:00,  2.54it/s]


Downloading: /Users/ineslaranjeira/Downloads/FlatIron/wittenlab/Subjects/ibl_witten_27/2021-01-20/001/alf/#2025-03-03#/_ibl_trials.stimOnTrigger_times.a143282d-f5fd-4c68-82c4-ddcf630e74cc.npy Bytes: 8256


100%|██████████| 0.00787353515625/0.00787353515625 [00:00<00:00,  2.85it/s]


Downloading: /Users/ineslaranjeira/Downloads/FlatIron/wittenlab/Subjects/ibl_witten_27/2021-01-20/001/alf/#2025-03-03#/_ibl_trials.stimOffTrigger_times.995f43d2-00ad-448a-a1b6-889b4a53f9cd.npy Bytes: 8256


100%|██████████| 0.00787353515625/0.00787353515625 [00:00<00:00,  1.95it/s]
100%|██████████| 0.07372283935546875/0.07372283935546875 [00:00<00:00,  4.49s/it]


Bugged sessions

In [ ]:
'4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b' # missing ME, but should be there according to alyx
'cf43dbb1-6992-40ec-a5f9-e8e838d0f643' # 
'4a45c8ba-db6f-4f11-9403-56e06a33dfa4'
'10fac7a1-919d-4ca7-83c4-4675bd8d9416'
'493170a6-fd94-4ee4-884f-cc018c17eeb9'
'edd22318-216c-44ff-bc24-49ce8be78374'
'73918ae1-e4fd-4c18-b132-00cb555b1ad2'
'25d1920e-a2af-4b6c-9f2e-fc6c65576544'
'eebacd5a-7dcd-4ba6-9dff-ec2a4d2f19e0'
'a4a74102-2af5-45dc-9e41-ef7f5aed88be'
'493170a6-fd94-4ee4-884f-cc018c17eeb9'